- 참고 : https://sieon-dev.tistory.com/15

## 1. 모듈 import (필요없는 모듈 삭제, 필요한 모듈 추가하면서 진행)
- pandas
- numpy
- matplotlib
- re
- konlpy
- tensorflow
- seaborn
- pandas_profiling

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pandas_profiling
import sklearn
from sklearn.model_selection import train_test_split
import collections
from collections import Counter
import exchange_calendars as ecals
import datetime
import time
from wordcloud import WordCloud
import matplotlib as mpl
import nltk

### 1-1. 파이썬 & 모듈 버전 확인 (필요없는 모듈 삭제, 필요한 모듈 추가하면서 진행)

In [ ]:
print(" python version : ", sys.version, "\n", "-" * 100)
print(" pandas version : ", pd.__version__, "\n", "-" * 100)
print(" numpy version : ", np.__version__, "\n", "-" * 100)
print(" re version : ", re.__version__, "\n", "-" * 100)
print(" tensorflow version : ", tensorflow.__version__, "\n", "-" * 100)
print(" seaborn version : ", sns.__version__, "\n", "-" * 100)
print(" pandas_profiling version : ", pandas_profiling.__version__, "\n", "-" * 100)
print(" sklearn version : ", sklearn.__version__, "\n", "-" * 100)
print(" exhange_calendars version : ", ecals.__version__, "\n", "-" * 100)
print(" nltk version : ", nltk.__version__, "\n", "-" * 100)

### 1-2. 컴퓨터 사양 확인

![](2022-03-24-13-31-45.png)

## 2. 데이터 불러오기

In [ ]:
# 뉴스데이터 불러오기
newsdata = pd.read_csv("../data/Newsfile3.csv", encoding='euc-kr')
newsdata.tail(3)

### 2-1. 휴장일 데이터 매칭

In [ ]:
# newsdata의 datatype 확인
newsdata.info()

- type 변환

In [ ]:
# datatype : int -> string
newsdata = newsdata.astype({'date':'string'})
newsdata.info()

In [ ]:
# datatype : string -> datetime
newsdata['date'] = pd.to_datetime(newsdata['date'])
newsdata.info()

- 뉴스데이터의 처음 날짜와 마지막 날짜 확인

In [ ]:
first_date = newsdata['date'][0]
last_date = newsdata['date'][-1:]
print(first_date, last_date)

In [ ]:
# newsdata -> newsdata_all : 처음 날짜와 마지막 날짜가 포함된 데이터로 이름 변결
newsdata_all = newsdata[newsdata['date'].between('2015-10-19', '2022-03-16')]
newsdata_all

- 리스트로 변환

In [ ]:
# 'date' 컬럼 리스트로 변경
news = newsdata_all['date'].to_list()
news[:5]

### 2-2. 한국 거래소 개장일 확인

In [ ]:
# 한국코드
XKRX = ecals.get_calendar("XKRX")       
# 2021-01-01은 개장일인지 확인 : 'False' -> 휴장일 , 'True' -> 개장일
print("2021-01-01 개장 확인: ", XKRX.is_session('2021-01-01'))    
# 오늘(22-03-24)은 개장일인지 확인 :'False' -> 휴장일 , 'True' -> 개장일
print("오늘날짜 개장일 확인: ", XKRX.is_session(datetime.date.today().strftime('%Y-%m-%d')))  
# 다음 개장일은 언제인지 확인
print("다음 개장일 날짜 확인: ", XKRX.next_open(pd.Timestamp.today().strftime('%Y-%m-%d')))

In [175]:
# 뉴스데이터 날짜 다음날에 해당하는 개장일 날짜 확인
next_open_list = []

for day in news:
    next_open_date = XKRX.next_open(day)
    next_open_list.append(next_open_date)

In [ ]:
next_open_list[:5]

In [ ]:
# 뉴스나온 날짜 다음 개장일 컬럼 추가
newsdata_all['NextOpenDate'] = next_open_list
newsdata_all.head(3)

In [ ]:
newsdata_all.info()

- type 변환

In [ ]:
newsdata_all = newsdata_all.astype({'NextOpenDate':'string'})
newsdata_all.info()

In [ ]:
a = newsdata_all['NextOpenDate'][0]
b = a.split(' ')[0]
b

In [ ]:
newsdata_all['NextOpenDate'] = newsdata_all['NextOpenDate'].apply(lambda x: x.split(' ')[0])
newsdata_all.head(3)

In [ ]:
newsdata_all['NextOpenDate'] = pd.to_datetime(newsdata_all['NextOpenDate'])
newsdata_all.info()

In [ ]:
# 개장일 기준 'Year' 컬럼 추가
newsdata_all['NextOpenDateYear'] = newsdata_all['NextOpenDate'].dt.year
newsdata_all.head(3)

In [ ]:
# col의 인덱스 변경
col = newsdata_all.columns.to_numpy()
col

In [ ]:
col = col[[0,3,2,1]]
col

In [ ]:
newsdata_all = newsdata_all[col]
newsdata_all.head(3)

## 3. 데이터 확인

In [ ]:
# 데이터 갯수 : 58442개
# 컬럼 갯수 : 4개
newsdata_all.info()

- 결측치 확인

In [ ]:
# 결측치 확인
newsdata_all.isnull().sum()

## 4. 기업별 '현재가', '당일종가-어제종가', '상승률', 'label' 데이터와 뉴스데이터 병합

### 4-1. 반도체 기업 데이터 불러오기 : 005290

In [ ]:
df1 = pd.read_excel('../data/반도체주가데이터/반도체주가데이터/덕산네오룩스(213420).xlsx')
df1.head(2)

In [ ]:
df1.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
df1.head(2)

- 긍정 & 부정 라벨링
    - 상승률이 0보다 크면 `1`로 매칭
    - 상승률이 0보다 작으면 `0`으로 매칭

In [ ]:
df1['label'] = df1.apply(lambda x: 1 if x['상승률'] >= 0 else 0, axis=1)
df1

- 라벨링 분포 확인

In [ ]:
sns.countplot(x='label', data=df1)

- data type 변경

In [ ]:
# data type : int -> string
df1 = df1.astype({'일자':'string'})
df1.info()

In [ ]:
# data type : string -> datetime
df1['일자'] = pd.to_datetime(df1['일자'])
df1.info()

### 4-2. 뉴스데이터 컬럼명 변경

In [ ]:
# 덕산네오룩스의 데이터프레임과 병합하기 위해 컬럼명 변경
newsdata_all.rename(columns={'NextOpenDate':'일자'}, inplace=True)
newsdata_all.head(2)

### 4-3. 뉴스데이터와 덕산네오룩스 주가데이터 병합

In [ ]:
merge_data = pd.merge(newsdata_all, df1, on='일자', how='inner')
merge_data.head(2)

- '일자', 'title', 'label'만 남긴 나머지 컬럼 삭제

In [ ]:
merge_data = merge_data[['일자', 'title', 'label']]
merge_data.head(5)

## 5. 데이터 전처리
    - 한글 이외의 문자열은 빈칸(`''`)처리
    - 품사 중 명사만 추출하여 'nouns' 컬럼에 삽입

In [ ]:
okt = Okt()
n_ = []
title_rename = []

for i in range(len(merge_data)):
    if (i % 10000 == 0):
        print(i, "단계 완료")
    title_rename.append(re.sub("[\(\[].*?[\)\]]", "", merge_data.iloc[i]['title']))
    n_.append(' '.join(okt.nouns(merge_data.iloc[i]['title'])))
merge_data['nouns'] = n_
merge_data['title'] = title_rename
merge_data = merge_data[merge_data['nouns'] != '']

# 2차 불용어 제거
merge_data['title'] = merge_data['title'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]','')
merge_data['title'].replace('', np.nan, inplace=True)
merge_data = merge_data.dropna(how='any')

In [ ]:
merge_data.head(3)

In [ ]:
news_labeling = merge_data.copy()

In [ ]:
# 결측치 확인
news_labeling.dropna(how='any', inplace=True)
news_labeling.shape

- label 분포 확인

In [ ]:
sns.countplot(x='label', data=news_labeling)

In [ ]:
# 라벨링 값 개수 확인
news_labeling['label'].value_counts()

## 6. train_data, test_data 분할

In [ ]:
# train, test set 분리
train_data, test_data = train_test_split(news_labeling)

### 6-1. train_data

In [ ]:
# train_data 확인
train_data.head(2)

In [ ]:
# train_data shape 확인
train_data.shape

### 6-2. test_data

In [ ]:
# test_data 확인
test_data.head(2)

In [ ]:
# test_data shape 확인
test_data.shape

## 7. 감성사전 구축

- 한 글자인 단어는 제외. 두 글자 이상인 단어들의 점수 초기화 진행

In [ ]:
vocab = {}
count = 0
for i in news_labeling['nouns']:
    i = i.split(' ')
    # news_labeling에 있는 데이터의 양만큼 순회
    for j in range(len(i)):
        # i 번째 행의 리스트에 있는 j번째 단어의 길이가 1 이하일 경우 count에 1 추가
        if i[j] in vocab or len(i[j]) <= 1:
            count += 1
            pass
        else:
            # i 번째 행의 리스트에 있는 j번째 단어의 길이가 1 이상일 경우 0
            vocab[i[j]] = 0

In [ ]:
vocab

- 상승비율과 하락비율을 정의해준 다음 라벨 값이 1이면 하락 비율을 각 단어에 더해주고 라벨값이 0이면 상승 비율을 차감
- 라벨값이 1이면 상승인데 왜 상승 비율을 더해주지 않았을지 : 라벨값이 1인 데이터가 아닌 데이터보다 훨씬 많다면 해당 단어들의 점수가 너무 커져서 점수가 고르지 못한 감성사전이 만들어지기 때문에 정규화했기 때문

In [ ]:
up = 29202
down = 29141

up_ratio = up/(up+down)
down_ratio = down/(up+down)

for i,w in enumerate(train_data['nouns']):
    w = w.split(' ')
    if (train_data.iloc[i]['label']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] - up_ratio

## 9. 워드 클라우드

### 9-1. train_data

In [ ]:
# train_data > nouns 컬럼 > 0번째 데이터의 단어 출력
train_data.nouns.to_list()[0]

In [ ]:
# train_data의 길이 확인
len(train_data.nouns.to_list())

- train_data의 'nouns' 데이터 합치기

In [ ]:
# 'nouns' 컬럼 데이터 리스트로 변환
nouns_list_train = []

for i in range(0, len(train_data.nouns.to_list())):
    noun = train_data.nouns.to_list()[i]
    nouns_list_train.append(noun)

In [ ]:
nouns_list_train[:5]

In [ ]:
nouns_list_train[0]

In [ ]:
nouns_list_train[0].split() + nouns_list_train[1].split()

In [ ]:
# WordCloud에 사용할 리스트 생성
nouns_total_train = []

for i in range(0, len(nouns_list_train)):
    nouns_total_train += nouns_list_train[i].split()

In [ ]:
nouns_total_train

In [ ]:
# 가장 많이 나온 단어 100개 저장
counts = Counter(nouns_total_train)
tags = counts.most_common(30000)
tags

- 한글 폰트 사용

In [ ]:
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'

- 필요없는 단어 제거 : stopwords 설정

In [ ]:
# 불용어 지정
stopwords = "램 사 액 첫 제 세 달 내년 반도체 분기 억 월 차 년 중 주 일 등 조 억원 용 익 위 개월 이재용 것 조원 초"
stop_list = stopwords.split()

# stop_list에 없는 단어만 추출 > 리스트를 만들 것
words_list = [word for word in nouns_total_train if word not in stop_list]

- 단어 빈도수 살펴보기

In [ ]:
words = nltk.Text(words_list)
plt.figure(figsize=(20,12))
# 많이 사용된 단어 보여주기
words.plot(100)        
plt.show()

In [ ]:
# 빈도수가 200 이상인 단어 10개
data = words.vocab().most_common(100)
data[:10]

In [ ]:
data[0][0]

- train_data 워드클라우드 생성

In [ ]:
wc = WordCloud(
    font_path='c:/Windows/Fonts/malgun.ttf',
    relative_scaling=0.2, background_color='white',
    colormap='coolwarm'
).generate_from_frequencies(dict(data))


In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis('off')
plt.show()

### 9-2. test_data

In [ ]:
# test_data > nouns 컬럼 > 0번째 데이터의 단어
test_data.nouns.to_list()[0]


# 'nouns' 컬럼 데이터 리스트로 변환
nouns_list_test = []

for i in range(0, len(test_data.nouns.to_list())):
    noun = test_data.nouns.to_list()[i]
    nouns_list_test.append(noun)


# 'nouns' 컬럼 데이터 단어 데이터 합치기
nouns_total_test = []

for i in range(0, len(nouns_list_test)):
    nouns_total_test += nouns_list_test[i]

# 가장 많이 나온 단어 100개 저장
counts = Counter(nouns_total_test)
tags = counts.most_common(30000)
tags


# 불용어 지정
stopwords = "올해 작년 램 사 액 첫 제 세 달 내년 반도체 분기 억 월 차 년 중 주 일 등 조 억원 용 익 위 개월 이재용 것 조원 초"
stop_list = stopwords.split()

# stop_list에 없는 단어만 추출 > 리스트를 만들 것
words_list = [word for word in nouns_total_test if word not in stop_list]

words = nltk.Text(words_list)
plt.figure(figsize=(20,12))
# 많이 사용된 단어 보여주기
words.plot(100)        
plt.show()

In [ ]:
# 빈도수가 200 이상인 단어 10개
data = words.vocab().most_common(200)
data[:10]

- test_data 워드클라우드 생성

In [ ]:
wc = WordCloud(
    font_path='c:/Windows/Fonts/malgun.ttf',
    relative_scaling=0.2, background_color='white',
    colormap='coolwarm'
).generate_from_frequencies(dict(data))

plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis('off')
plt.show()